In [1]:
# Dependencies
# ----------------------------------
import pandas as pd

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date, DateTime

from sqlalchemy import create_engine, ForeignKey

Base = declarative_base()

# from sqlalchemy.types import Boolean, Date, DateTime, Float, Integer, Text, Time, Interval

In [2]:
msa_df = pd.read_csv('../data/final_msa.csv', index_col=0)
msa_df.head()

,incident_id,incident_date,state,city_county,address,number_killed,number_injured,latitude,longitude
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4,43.403320,-83.947553
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4,39.972228,-83.005167
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3,40.015002,-75.070959
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3,39.278954,-76.642987
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5,41.878778,-87.679500


In [3]:
msa_df['incident_date'] =  pd.to_datetime(msa_df['incident_date'])

In [4]:
msa_df.dtypes

incident_id                int64
incident_date     datetime64[ns]
state                     object
city_county               object
address                   object
number_killed              int64
number_injured             int64
latitude                 float64
longitude                float64
dtype: object

In [5]:
# class Directors(Base):
#     __tablename__ = 'directors'
#     director_id = Column(String, primary_key=True)
#     director_name = Column(String(255))

In [6]:
# Create Classes
class Msa(Base):
    __tablename__ = 'msa'
    incident_id = Column(Integer, primary_key=True)
    incident_date = Column(Date)
    state = Column(String(255))
    city = Column(String(255))
    address = Column(String(255))
    killed = Column(Integer)
    injured = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)

class Guns(Base):
    __tablename__ = 'ownership_data'
    index = Column(Integer, primary_key = True)
    state = Column(String(255))
    number_per_capita = Column(Float)
    number_registered = Column(Integer)
    
class Poverty(Base):
    __tablename__ = 'poverty_data'
    rank = Column(Integer)
    state = Column(String(255), primary_key = True)
    poverty = Column(Float)

class Election(Base):
    __tablename__ = 'election_data'
    state = Column(String(255), primary_key = True)
    result = Column(String(255))
    
class State(Base):
    __tablename__ = 'state_data'
    state = Column(String(255), primary_key = True)
    latitude = Column(Float)
    longitude = Column(Float)

In [7]:
# These tables only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'msa': Table('msa', MetaData(bind=None), Column('incident_id', Integer(), table=<msa>, primary_key=True, nullable=False), Column('incident_date', Date(), table=<msa>), Column('state', String(length=255), table=<msa>), Column('city', String(length=255), table=<msa>), Column('address', String(length=255), table=<msa>), Column('killed', Integer(), table=<msa>), Column('injured', Integer(), table=<msa>), Column('latitude', Float(), table=<msa>), Column('longitude', Float(), table=<msa>), schema=None), 'ownership_data': Table('ownership_data', MetaData(bind=None), Column('index', Integer(), table=<ownership_data>, primary_key=True, nullable=False), Column('state', String(length=255), table=<ownership_data>), Column('number_per_capita', Float(), table=<ownership_data>), Column('number_registered', Integer(), table=<ownership_data>), schema=None), 'poverty_data': Table('poverty_data', MetaData(bind=None), Column('rank', Integer(), table=<poverty_data>), Column('state', String(l

In [8]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../data/msa.sqlite")
conn = engine.connect()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
# Note that adding to the session does not update the table. It queues up those queries.
# Adding data to directors table from directors csv file
for index, name in msa_df.iterrows():
    session.add(Msa(incident_id=name['incident_id'], incident_date=name['incident_date'], state=name['state'], \
                   city=name['city_county'], address=name['address'], killed=name['number_killed'], \
                   injured=name['number_injured'], latitude=name['latitude'], \
                   longitude=name['longitude']
                   ))

In [12]:
session.new

IdentitySet([<__main__.Msa object at 0x00000213286037F0>, <__main__.Msa object at 0x00000213286037B8>, <__main__.Msa object at 0x0000021328603860>, <__main__.Msa object at 0x00000213286038D0>, <__main__.Msa object at 0x0000021328603940>, <__main__.Msa object at 0x00000213286039B0>, <__main__.Msa object at 0x0000021328603A20>, <__main__.Msa object at 0x0000021328603A90>, <__main__.Msa object at 0x0000021328603B00>, <__main__.Msa object at 0x0000021328603B70>, <__main__.Msa object at 0x0000021328603BE0>, <__main__.Msa object at 0x0000021328603C50>, <__main__.Msa object at 0x0000021328603CC0>, <__main__.Msa object at 0x0000021328603D30>, <__main__.Msa object at 0x0000021328603DA0>, <__main__.Msa object at 0x0000021328603E10>, <__main__.Msa object at 0x0000021328603E80>, <__main__.Msa object at 0x0000021328603EF0>, <__main__.Msa object at 0x0000021328603F60>, <__main__.Msa object at 0x0000021328603FD0>, <__main__.Msa object at 0x0000021328681080>, <__main__.Msa object at 0x00000213286810F0

In [13]:
session.commit()

In [ ]:
# session.rollback()

In [14]:
guns_df = pd.read_csv('../data/gun_ownership_clean.csv')
guns_df.head()

,state,number_per_capita,number_registered
0,Wyoming,229.24,"132,806"
1,District of Columbia,68.05,"47,228"
2,New Hampshire,46.76,"64,135"
3,New Mexico,46.73,"97,580"
4,Virginia,36.34,"307,822"


In [15]:
guns_df.dtypes

state                 object
number_per_capita    float64
number_registered     object
dtype: object

In [16]:
# Remove comma from number registered column
guns_df['number_registered'] = guns_df['number_registered'].str.replace(',', '')

#Change data type to integer
guns_df['number_registered'] = guns_df['number_registered'].astype(int)

guns_df.dtypes

state                 object
number_per_capita    float64
number_registered      int32
dtype: object

In [17]:
for index, name in guns_df.iterrows():
    session.add(Guns(state=name['state'], number_per_capita=name['number_per_capita'], \
                   number_registered=name['number_registered']))

In [18]:
session.commit()

In [19]:
poverty_df = pd.read_csv('../data/poverty_data_clean.csv')
poverty_df.head()

,rank,state,poverty
0,1,New Hampshire,7.3
1,2,Utah,8.9
2,3,Maryland,9.0
3,4,Minnesota,9.0
4,5,New Jersey,9.2


In [20]:
poverty_df.dtypes

rank         int64
state       object
poverty    float64
dtype: object

In [21]:
for index, name in poverty_df.iterrows():
    session.add(Poverty(rank=name['rank'], state=name['state'], poverty=name['poverty']))

In [22]:
session.commit()

In [23]:
election_df = pd.read_csv('../data/election_data_clean.csv')
election_df.head()

,state,result
0,Alabama,blue
1,Alaska,red
2,Arizona,blue
3,Arkansas,red
4,California,red


In [24]:
election_df.dtypes

state     object
result    object
dtype: object

In [25]:
for index, name in election_df.iterrows():
    session.add(Election(state=name['state'], result=name['result']))

In [26]:
session.commit()

In [27]:
state_df = pd.read_csv('../data/state_clean.csv')
state_df.head()

,state,latitude,longitude
0,Alaska,63.588753,-154.493062
1,Alabama,32.318231,-86.902298
2,Arkansas,35.201050,-91.831833
3,Arizona,34.048928,-111.093731
4,California,36.778261,-119.417932


In [28]:
state_df.dtypes

state         object
latitude     float64
longitude    float64
dtype: object

In [29]:
for index, name in state_df.iterrows():
    session.add(State(state=name['state'], latitude=name['latitude'], longitude=name['longitude']))

In [30]:
session.commit()

In [31]:
session.close()